<a href="https://colab.research.google.com/github/chdmitr2/Deep-Learning-22961/blob/main/maman11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Maman 11**



**Question 1**


**A. Broadcast one tensor A to the shape of another tensor B**


We want a function broadcast_A_to_B(A, B) that attempts to broadcast A so that its shape matches B. We will do the following:

1.   Check if broadcasting is possible using the rules described in Part B (i.e., compare dimensions from right to left).
2.  If it is possible, insert (unsqueeze) dimensions of size 1 on the left of A as needed.
3.  Foor each dimension where B has a size greater than 1, but A has a size of 1, we need to manually duplicate  A along that dimension.Since we cannot use .expand() or .repeat(), we will do this by concatenating multiple copies of A along that dimension. This way, A will match the shape of B


In [ ]:
import torch

# Helper function to replicate a tensor along a given dimension.
# We cannot use x.repeat, so we manually cat copies of x.
def replicate_dim(x, dim, times):
    if times <= 1:
        return x  # no replication needed if times=1
    # We build a list of cloned tensors, then concatenate along `dim`.
    clones = [x.clone() for _ in range(times)]
    return torch.cat(clones, dim=dim)


def broadcast_A_to_B(A, B):
    """
    Broadcast tensor A to the shape of B, returning the broadcasted version
    of A. Raises ValueError if it is impossible to broadcast.
    """
    #  1. Check broadcastability (rely on a function from Part B, shown below).
    can_broad, explanation = can_broadcast(A, B)
    if not can_broad:
        raise ValueError(
            f"Cannot broadcast A of shape {A.shape} to B of shape {B.shape}. Reason: {explanation}"
        )

    #  2. Figure out how many dims to pad on the left
    sA = list(A.shape)
    sB = list(B.shape)
    while len(sA) < len(sB):
        sA.insert(0, 1)  # pad left
    # We also want to unsqueeze in the actual tensor A
    A_broad = A.clone()
    while A_broad.dim() < len(sB):
        A_broad = A_broad.unsqueeze(0)

    #  3. Physically replicate A_broad in each dimension if needed
    for dim in range(len(sB)):
        sizeA = A_broad.size(dim)
        sizeB = sB[dim]
        if sizeA == 1 and sizeB > 1:
            # replicate along this dimension
            A_broad = replicate_dim(A_broad, dim, sizeB)
        # if sizeA == sizeB, do nothing, it already matches

    return A_broad


**B. Check if two tensors A and B can be broadcast together**

We define a helper can_broadcast(A, B) that returns (True, explanation) if the two shapes are compatible under broadcasting rules, or (False, explanation) if they are not. In the explanation, we can also include the final broadcast shape if it is valid.

Broadcast rules :

1.   Compare the dimensions from the rightmost (last) going left.
2.   At each dimension:
    *   If the two sizes are equal, that dimension is fine.
    *   Else if one of them is 1 and the other is > 1, that is also fine (the 1 can broadcast).
    *   Otherwise, it is not possible to broadcast.
    

In [ ]:
def can_broadcast(A, B):
    """
    Checks if A and B can be broadcast together.
    Returns (True, explanation) or (False, explanation).
    """
    sA = list(A.shape)
    sB = list(B.shape)
    # Pad the shorter shape with 1 on the left
    while len(sA) < len(sB):
        sA.insert(0, 1)
    while len(sB) < len(sA):
        sB.insert(0, 1)

    # Check dimension by dimension
    for dimA, dimB in zip(sA, sB):
        if not (dimA == dimB or dimA == 1 or dimB == 1):
            return False, f"Dimension mismatch ({dimA} vs {dimB})"
    # If we haven't returned False, they are broadcastable
    # The final shape after broadcast:
    final_shape = [max(dimA, dimB) for dimA, dimB in zip(sA, sB)]
    return True, f"Broadcast is possible. Final shape: {final_shape}"


**C. Joint broadcast of two tensors**

Now we write a function that, given two tensors A and B, returns both of them in their “joint broadcasted” shape (the same shape for both). This is basically what torch.broadcast_tensors(A,B) does. We will:


1.  Check if A and B can be broadcast (using our can_broadcast).
2.  Compute the final broadcast shape.
3.  Use the “broadcast_A_to_B” logic (or a similar approach) to broadcast each one to the final shape.
4.  Return (A_broadcasted, B_broadcasted).

In [ ]:
def my_broadcast_tensors(A, B):
    """
    Returns a tuple of two new tensors (A_broad, B_broad) that both have
    the same broadcasted shape, if possible.
    """
    # 1. Check broadcast feasibility
    can_broad, explanation = can_broadcast(A, B)
    if not can_broad:
        raise ValueError(f"Cannot broadcast shapes {A.shape} and {B.shape}. {explanation}")

    # 2. Derive the final broadcast shape from the explanation text or by re-computing
    sA = list(A.shape)
    sB = list(B.shape)
    while len(sA) < len(sB):
        sA.insert(0, 1)
    while len(sB) < len(sA):
        sB.insert(0, 1)

    # final broadcast shape = elementwise max
    final_shape = [max(a, b) for a, b in zip(sA, sB)]

    # 3. Broadcast each separately to final_shape
    A_out = broadcast_to_shape(A, final_shape)  # see below
    B_out = broadcast_to_shape(B, final_shape)

    return A_out, B_out


def broadcast_to_shape(X, desired_shape):
    """
    Helper that broadcasts X to 'desired_shape' using the same replicate approach.
    """
    # 1. Check if X can be broadcast to desired_shape
    #    We can do a quick shape-compatibility check:
    sX = list(X.shape)
    while len(sX) < len(desired_shape):
        sX.insert(0, 1)

    for (dimX, dimWant) in zip(reversed(sX), reversed(desired_shape)):
        # compare from right to left
        if not (dimX == dimWant or dimX == 1 or dimWant == 1):
            raise ValueError(f"Shape mismatch: cannot match {X.shape} to {desired_shape}.")

    # 2. Actually replicate data as needed
    X_broad = X.clone()
    # unsqueeze if needed
    while X_broad.dim() < len(desired_shape):
        X_broad = X_broad.unsqueeze(0)

    # Now replicate in each dimension
    for dim in range(len(desired_shape)):
        sizeX = X_broad.size(dim)
        sizeD = desired_shape[dim]
        if sizeX == 1 and sizeD > 1:
            X_broad = replicate_dim(X_broad, dim, sizeD)
        # If they are equal, do nothing

    return X_broad


**D. Demonstration & Comparison with built-in PyTorch**

Below we create some example tensors of various shapes and compare the results of our “manual” broadcasting versus PyTorchs built-in broadcasting (e.g. by doing an elementwise operation).We can use normal PyTorch broadcasting in typical expressions (like A + B) just to check that the final shapes/values match.

In [ ]:
# Let's define some example shapes:
examples = [
    (torch.rand(3),      torch.rand(3)),
    (torch.rand(1,5),    torch.rand(3,1,5)),
    (torch.rand(2,1,4),  torch.rand(2,3,4)),
    (torch.rand(1,1),    torch.rand(2,3)),
]

for i, (A, B) in enumerate(examples, 1):
    print(f"\n--- Example {i} ---")
    print("A.shape =", A.shape, "B.shape =", B.shape)

    # A. Using our function from Part A: broadcast A to shape of B
    try:
        A_to_B = broadcast_A_to_B(A, B)
        print("Shape of broadcast_A_to_B(A,B) =", A_to_B.shape)
    except ValueError as e:
        print("Error in broadcast_A_to_B:", e)

    # B. Check broadcast
    can_broad, expl = can_broadcast(A, B)
    print("can_broadcast(A,B) =", can_broad, "|", expl)

    # C. Joint broadcast
    try:
        A_broad, B_broad = my_broadcast_tensors(A, B)
        print("Shapes from my_broadcast_tensors: A_broad:", A_broad.shape,
              "B_broad:", B_broad.shape)
    except ValueError as e:
        print("Error in my_broadcast_tensors:", e)

    # Compare with PyTorch's normal broadcasting by doing an elementwise op
    try:
        C_pytorch = A + B  # triggers normal PyTorch broadcasting internally
        print("PyTorch result shape (A + B):", C_pytorch.shape)
        # We can check that A_broad + B_broad matches this as well:
        check = (A_broad + B_broad).shape == C_pytorch.shape
        print("Check shapes match our broadcast vs PyTorch:", check)
    except RuntimeError as e:
        print("PyTorch broadcasting error (which should match ours):", e)



--- Example 1 ---
A.shape = torch.Size([3]) B.shape = torch.Size([3])
Shape of broadcast_A_to_B(A,B) = torch.Size([3])
can_broadcast(A,B) = True | Broadcast is possible. Final shape: [3]
Shapes from my_broadcast_tensors: A_broad: torch.Size([3]) B_broad: torch.Size([3])
PyTorch result shape (A + B): torch.Size([3])
Check shapes match our broadcast vs PyTorch: True

--- Example 2 ---
A.shape = torch.Size([1, 5]) B.shape = torch.Size([3, 1, 5])
Shape of broadcast_A_to_B(A,B) = torch.Size([3, 1, 5])
can_broadcast(A,B) = True | Broadcast is possible. Final shape: [3, 1, 5]
Shapes from my_broadcast_tensors: A_broad: torch.Size([3, 1, 5]) B_broad: torch.Size([3, 1, 5])
PyTorch result shape (A + B): torch.Size([3, 1, 5])
Check shapes match our broadcast vs PyTorch: True

--- Example 3 ---
A.shape = torch.Size([2, 1, 4]) B.shape = torch.Size([2, 3, 4])
Shape of broadcast_A_to_B(A,B) = torch.Size([2, 3, 4])
can_broadcast(A,B) = True | Broadcast is possible. Final shape: [2, 3, 4]
Shapes from m